# 1. Preparing Invironment
___
___

In [1]:
pip install dlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python-headless


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install  face_recognition

Note: you may need to restart the kernel to use updated packages.


# 2. Importing Libraries
___
___

In [25]:
import cv2
import face_recognition
import os
import pickle
from datetime import datetime
import time


# 3. Loading Haarcascade Model
___

In [26]:
face_detection = cv2.CascadeClassifier(r'c:\Users\skynet\Pictures\Ml Projects\haarcascade_frontalface_default.xml')

# 4. Database Setup and Load Encodings
___

In [27]:
encoding_file = 'encodings.pkl'
log_file_path = 'attendance_log.txt'


## 4.1  create a new list if  encoding file not available
___

In [28]:
if os.path.exists(encoding_file):
    with open(encoding_file, 'rb') as f:
        known_encodings = pickle.load(f)
else:
    known_encodings = []

# 5. Start the Camera and Open Log File
___

In [29]:
camera = cv2.VideoCapture(0)

# 6.Track the presence status (in/out)
___


In [30]:
log_file = open(log_file_path, 'a')

person_present = {}
last_seen_time = {}
cooldown_time = 20    # to confirm that you're out


# 7.Loop for Face Detection and Attendance Logging
___

In [31]:
while True:
   
    st, frame = camera.read()
    if not st:
        print("Failed to capture image")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_detection.detectMultiScale(gray_frame, 1.3, 5)

    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y, x+w, y+h, x)])

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces([encoding[1] for encoding in known_encodings], face_encoding, tolerance=0.6)
            face_distances = face_recognition.face_distance([encoding[1] for encoding in known_encodings], face_encoding)

            best_match_index = None
            if len(face_distances) > 0:
                best_match_index = face_distances.argmin()

            if best_match_index is not None and matches[best_match_index]:
                person_id = known_encodings[best_match_index][0]
                cv2.putText(frame, f"Known Person {person_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                if person_id not in person_present:
                    log_file.write(f"{current_time}: Person {person_id} entered\n")
                    person_present[person_id] = True

                last_seen_time[person_id] = time.time()

            else:
                person_id = f"Person_{len(known_encodings) + 1}"
                known_encodings.append((person_id, face_encoding))
                cv2.putText(frame, "New Person", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
                log_file.write(f"{current_time}: New person detected and added as {person_id}\n")
                person_present[person_id] = True
                last_seen_time[person_id] = time.time()
            
            
    for person_id in list(person_present.keys()):
        if time.time() - last_seen_time.get(person_id, 0) > cooldown_time:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S') 
            log_file.write(f"{current_time}: Person {person_id} left\n")
            del person_present[person_id] 

    cv2.imshow('Camera', frame)

    if cv2.waitKey(33) & 0xff == ord('q'):
        break




# 8.Release Camera and Save Encodings
___

In [32]:

camera.release()
cv2.destroyAllWindows()

with open(encoding_file, 'wb') as f:
    pickle.dump(known_encodings, f)


log_file.close()
